In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d jangedoo/utkface-new

100% 330M/331M [00:10<00:00, 43.2MB/s]
100% 331M/331M [00:10<00:00, 34.0MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/utkface-new.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [5]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [6]:
age = []
gender = []
img_path = []

for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [8]:
len(age)

23708

In [10]:
len(gender)

23708

In [11]:
len(img_path)

23708

In [13]:
df = pd.DataFrame({'Age': age, 'Gender': gender, 'img' : img_path})

In [14]:
df.shape

(23708, 3)

In [15]:
df.head()

,Age,Gender,img
0,1,1,1_1_2_20161219205734821.jpg.chip.jpg
1,1,0,1_0_3_20161219225759904.jpg.chip.jpg
2,1,1,1_1_2_20161219190719971.jpg.chip.jpg
3,22,0,22_0_0_20170117135052479.jpg.chip.jpg
4,29,1,29_1_1_20170112211222837.jpg.chip.jpg


In [16]:
df.isnull().sum()

Age       0
Gender    0
img       0
dtype: int64

In [17]:
# split the data into train and test

train_df = df.sample(frac=1, random_state=0).iloc[:20000]
test_df = df.sample(frac=1, random_state=0).iloc[20000:]

In [19]:
print(train_df.shape)
print()
print(test_df.shape)

(20000, 3)

(3708, 3)


In [21]:
train_datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=40,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    interpolation_order=1)

test_datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=40,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0)

In [22]:
train_generator = train_datagen.flow_from_dataframe(train_df, directory = folder_path,x_col ='img',
                                                    y_col=['Age', 'Gender'], target_size=(200,200),
                                                    class_mode='multi_output')


test_generator = test_datagen.flow_from_dataframe(test_df, directory = folder_path,x_col ='img',
                                                    y_col=['Age', 'Gender'], target_size=(200,200),
                                                    class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


# Transfer Learning Model